# Global Imports

In [ ]:
import os
import pprint
import random as rnd
import sys
from pathlib import Path as Pt
from time import sleep
from typing import Dict, Iterator, List, Optional, Set, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv
from matplotlib import gridspec
from numpy import random as nrnd

from utils_002 import Template, gen_xyz, load_cfg, prep_cuboid, read_large_file_lines

In [ ]:
def setup_plotter(grid: pv.RectilinearGrid, dims: Tuple[int, int, int]) -> pv.Plotter:
    """Initialize and configure the PyVista Plotter."""
    view_vectors = {
        "XZ": {"vector": (0, -1, 0), "viewup": (0, 0, 1)},  # XZ to Camera
        "YZ": {"vector": (1, 0, 0), "viewup": (0, 0, 1)},  # YZ to Camera
        "XY": {"vector": (0, 0, 1), "viewup": (0, 1, 0)},  # XY to Camera
        "YX": {"vector": (0, 0, -1), "viewup": (1, 0, 0)},  # YX to Camera
    }

    profiles = {
        "P_001_1": {
            "w_size_koef": {"kw": 1.0, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": 15,
                "Elevation": 20,
                "Roll": 0,
                "shift": [-5.0, -5.0, -4.0],
                "Zoom": 1.15,
                "parallel_projection": False,
            },
        },
        "P_001_2": {
            "w_size_koef": {"kw": 1.5, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 2.5},
            "font_size": 14,
            "camera": {
                "Azimuth": 0,
                "Elevation": 0,
                "Roll": 0,
                "shift": [0, 0, 0],
                "Zoom": 1.0,
                "parallel_projection": False,
            },
        },
        "P_002_1": {
            "w_size_koef": {"kw": 1.0, "kh": 1.0},
            "view_vector": "YZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": 15,
                "Elevation": 20,
                "Roll": 0,
                "shift": [5.0, -5.0, -4.0],
                "Zoom": 1.15,
                "parallel_projection": False,
            },
        },
        "P_002_2": {
            "w_size_koef": {"kw": 1.5, "kh": 1.0},
            "view_vector": "YZ",
            "light": {"dist": 100, "intensity": 2.5},
            "font_size": 14,
            "camera": {
                "Azimuth": 0,
                "Elevation": 0,
                "Roll": 0,
                "shift": [0, 0, 0],
                "Zoom": 1.0,
                "parallel_projection": False,
            },
        },
        "P_003_1": {
            "w_size_koef": {"kw": 1.0, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": -45,
                "Elevation": 20,
                "Roll": 0,
                "shift": [-4.0, 1.0, -4.0],
                "Zoom": 1.05,
                "parallel_projection": False,
            },
        },
        "P_003_2": {
            "w_size_koef": {"kw": 1.5, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 2.5},
            "font_size": 14,
            "camera": {
                "Azimuth": 0,
                "Elevation": 0,
                "Roll": 0,
                "shift": [0, 0, 0],
                "Zoom": 1.0,
                "parallel_projection": False,
            },
        },
        "P_003_3": {  # Для 50x100x50
            "w_size_koef": {"kw": 1.5, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": -45,
                "Elevation": 20,
                "Roll": 0,
                "shift": [-6.0, -8.0, -6.0],
                "Zoom": 1.4,
                "parallel_projection": False,
            },
        },
        "P_003_4": {  # Для 50x200x50
            "w_size_koef": {"kw": 1.5, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": -45,
                "Elevation": 20,
                "Roll": 0,
                "shift": [-4.0, -10.0, -10.0],
                "Zoom": 1.65,
                "parallel_projection": False,
            },
        },
        "P_004_1": {
            "w_size_koef": {"kw": 1.0, "kh": 1.0},
            "view_vector": "XZ",
            "light": {"dist": 100, "intensity": 1.0},
            "font_size": 18,
            "camera": {
                "Azimuth": -45,
                "Elevation": 10,
                "Roll": 0,
                "shift": [-4.0, 1.0, -4.0],
                "Zoom": 1.05,
                "parallel_projection": False,
            },
        },
    }
    profile = profiles["P_003_1"]

    w_size_base = {"w": 640, "h": 640}
    w_size_koef = profile["w_size_koef"]
    window_size = (
        round(w_size_base["w"] * w_size_koef["kw"]),
        round(w_size_base["h"] * w_size_koef["kh"]),
    )

    sx, sy, sz = dims
    center = np.array([sx, sy, sz]) / 2

    pl = pv.Plotter(
        off_screen=True,
        window_size=window_size,
        line_smoothing=True,
        point_smoothing=True,
        polygon_smoothing=True,
    )
    pl.set_background("lightblue")
    pl.enable_anti_aliasing("msaa", multi_samples=64)

    # add six colored positional lights along axes
    light = profile["light"]
    axis_colors = {
        "X": {"ID": 0, "color": "#00F500"},
        "Y": {"ID": 1, "color": "#0000F5"},
        "Z": {"ID": 2, "color": "#F50000"},
    }
    for axis, id_and_color in axis_colors.items():
        ID, color = id_and_color.values()
        for sign in [1, -1]:
            pos = center.copy()
            pos[ID] *= light["dist"] * sign
            pl.add_light(
                pv.Light(
                    position=tuple(pos),
                    light_type="scene light",
                    color=color,
                    intensity=light["intensity"],
                    positional=True,
                )
            )

    # add semi‑transparent grid mesh
    mesh = grid.threshold(0.0, scalars="mask")
    pl.add_mesh(mesh, color="green", opacity=0.16, lighting=False, name="grid")

    # draw bounds
    pl.show_bounds(
        grid="back",
        all_edges=True,
        xtitle="X",
        ytitle="Y",
        ztitle="Z",
        font_size=profile["font_size"],
        use_3d_text=False,
        location="outer",
    )

    # set camera view: XZ plane, slight rotation & shift
    view_vector = view_vectors[profile["view_vector"]]
    pl.view_vector(vector=view_vector["vector"], viewup=view_vector["viewup"])

    camera_cfg = profile["camera"]

    if camera_cfg["parallel_projection"]:
        pl.enable_parallel_projection()

    pl.camera.Azimuth(camera_cfg["Azimuth"])
    pl.camera.Elevation(camera_cfg["Elevation"])
    pl.camera.Roll(camera_cfg["Roll"])
    shift = np.array(camera_cfg["shift"])
    pl.camera.position = tuple(np.array(pl.camera.position) + shift)
    pl.camera.focal_point = tuple(np.array(pl.camera.focal_point) + shift)

    pl.camera.Zoom(camera_cfg["Zoom"])

    # remove initial grid actor name for clarity
    pl.remove_actor("grid")

    return pl


def render_state(
    pl: pv.Plotter,
    grid: pv.RectilinearGrid,
    mask_3d: np.ndarray,
    coords: Tuple[np.ndarray, np.ndarray, np.ndarray],
    state: np.ndarray,
    out_path: Pt,
) -> None:
    """
    Оновлює маску та сітку для заданого стану і зберігає скріншот.

    Args:
        pl (pv.Plotter): Об'єкт PyVista Plotter, що містить сцену.
        grid (pv.RectilinearGrid): Прямокутна сітка PyVista, що представляє 3D-простір.
        mask_3d (np.ndarray): 3D-масив NumPy (цілі числа), який буде оновлено
                              для відображення активних комірок.
        coords (Tuple[np.ndarray, np.ndarray, np.ndarray]): Кортеж з трьох 1D-масивів
                                                            (x, y, z) індексів активних комірок.
        state (np.ndarray): 1D-масив NumPy, що вказує стан кожної комірки (1 - активна).
        out_path (Path): Шлях для збереження вихідного скріншоту.
    """
    x_coords, y_coords, z_coords = coords

    mask = state == 1

    pl.remove_actor("active_mesh")

    if mask.any():
        mask_3d.fill(0)

        mask_3d[x_coords[mask], y_coords[mask], z_coords[mask]] = 1

        grid.cell_data["mask"] = mask_3d.ravel(order="F")

        active_mesh = grid.threshold(0.5, scalars="mask")
        pl.add_mesh(
            active_mesh,
            color="white",
            show_edges=True,
            edge_color="black",
            line_width=1,
            opacity=1.0,
            lighting=True,
            ambient=0.05,
            diffuse=0.4,
            name="active_mesh",
        )

    pl.screenshot(str(out_path))


def process_directory(
    dir_path: Pt, plot_last_only: bool = True
) -> Tuple[Dict, Optional[np.ndarray]]:
    """
    Обрабатывает папку симуляции:
      - читает InitSettings.ini и TimeStates.txt построчно,
      - рисует каждое состояние (или только первое/последнее),
      - возвращает конфигурацию и последний массив состояния.

    Args:
        dir_path: путь к директории симуляции.
        plot_last_only: если False — рендерит все состояния в subfolder render_001.

    Returns:
        (cfg_dict, last_state) — словарь конфигурации и numpy-массив последнего состояния.
    """
    dir_path = Pt(dir_path)
    cfg_file = dir_path / "InitSettings.ini"
    states_file = dir_path / "TimeStates.txt"

    if not (cfg_file.exists() and states_file.exists()):
        raise FileNotFoundError(f"Missing files in {dir_path}")

    # если нужно рисовать все — создаём папку
    out_all = dir_path / "render_001"
    if not plot_last_only:
        out_all.mkdir(exist_ok=True)

    # 1) загрузка конфигурации
    cfg = load_cfg(path_to_config=cfg_file)
    run_id = dir_path.name.split("_", 1)[0]
    cfg["id"] = run_id

    sx, sy, sz = cfg["Sx"], cfg["Sy"], cfg["Sz"]
    # 2) подготовка координат и маски
    x, y, z = gen_xyz(sx, sy, sz, mode=2)
    mask_3d = np.zeros((sx, sy, sz), dtype=np.uint8)

    # 3) инициализация сетки и плоттера
    grid = pv.RectilinearGrid(np.arange(sx + 1), np.arange(sy + 1), np.arange(sz + 1))
    grid.cell_data["mask"] = np.zeros(x.size, dtype=np.uint8)
    pl = setup_plotter(grid, dims=(sx, sy, sz))

    first_state: Optional[np.ndarray] = None
    last_state: Optional[np.ndarray] = None

    # 4) построчное чтение и рендер
    for idx, line in enumerate(read_large_file_lines(states_file)):
        state = np.fromstring(line, dtype=np.uint8, sep=":")

        if first_state is None:
            first_state = state
        last_state = state

        if not plot_last_only:
            out_path = out_all / f"{run_id}_state_{idx:08}.png"
            render_state(pl, grid, mask_3d, (x, y, z), state, out_path)

    # 5) всегда рендерим первый и последний (если они есть)
    if first_state is not None:
        render_state(
            pl,
            grid,
            mask_3d,
            (x, y, z),
            first_state,
            dir_path / f"{run_id}_state_first.png",
        )
        # если есть второй distinct state или только one
        if last_state is not None:
            render_state(
                pl,
                grid,
                mask_3d,
                (x, y, z),
                last_state,
                dir_path / f"{run_id}_state_last.png",
            )

    pl.close()
    return cfg, last_state


base_folder = Pt(  #
    r"1754163803010260_Mode2_N10_X50Y200Z50_T3e2_C9.58767e-8_Nt1e11_Pb0.05"
    #
)

try:
    for root, _, files in os.walk(base_folder):
        if {"InitSettings.ini", "TimeStates.txt"}.issubset(files):
            print(f"Processing directory: {root}")
            cfg, state = process_directory(Pt(root), plot_last_only=False)
except Exception as e:
    print(f"Error in main loop: {str(e)}")
    raise